# IMPACT paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import selectionStrategy
from liriscat import CDM

import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload
import IMPACT

#### 1.2. Set up the loggers (recommended)

In [2]:
utils.setuplogger(verbose = True, log_name="liriscat")

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(selectionStrategy)
reload(CDM)
reload(dataset)

<module 'liriscat.dataset' from '/home/arthurb/Programmation/liriscat/liriscat/dataset/__init__.py'>

In [4]:
config = utils.generate_eval_config(esc = 'error', valid_metric= 'mi_acc', pred_metrics = ['rmse', 'mi_acc'], profile_metrics = ['doa', 'pc-er', 'rm'], save_params=False, n_query=4, num_epochs=4, batch_size=1024)
utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 00:18] math2


In [5]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'+str(config["dataset_name"])
config['params_path']='../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))

[INFO 00:18] #### math2 ####
[INFO 00:18] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': False, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.02026, 'batch_size': 1024, 'num_epochs': 4, 'eval_freq': 1, 'patience': 30, 'device': device(type='cuda'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['rmse', 'mi_acc'], 'profile_metrics': ['doa', 'pc-er', 'rm'], 'num_responses': 12, 'low_mem': False, 'n_query': 4, 'CDM': 'impact', 'd_in': 4} ####


In [6]:
i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})

In [7]:
reload(dataset)
train_data = dataset.CATDataset(train_df, concept_map, metadata, config)
valid_data = dataset.CATDataset(valid_df, concept_map, metadata, config)
test_data = dataset.CATDataset(test_df, concept_map, metadata, config)

/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:134: UserWarning: The torch_array contains both query and the meta set !
  warnings.warn("The torch_array contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:139: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")


In [9]:
reload(utils)
reload(selectionStrategy)
reload(dataset)
reload(CDM)

S = selectionStrategy.Random(**config)
S.train(train_data, valid_data)

RandomModel()
[INFO 05:13] train on cuda
[INFO 05:13] -- START Training --


/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:139: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/selectionStrategy/random.py:37: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:144: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")
  0%|          | 0/4 [00:00<?, ?it/s]

[INFO 05:19] t : 0, size : torch.Size([2508])
[INFO 05:19] t : 1, size : torch.Size([5046])
[INFO 05:19] t : 2, size : torch.Size([7560])
[INFO 05:19] t : 3, size : torch.Size([10038])
[INFO 05:19] 0.47061145305633545, 1.511189579963684


 25%|██▌       | 1/4 [00:06<00:19,  6.48s/it]/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:144: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")


[INFO 05:25] t : 0, size : torch.Size([2515])
[INFO 05:25] t : 1, size : torch.Size([4973])
[INFO 05:25] t : 2, size : torch.Size([7458])
[INFO 05:25] t : 3, size : torch.Size([9940])
[INFO 05:26] 0.47347623109817505, 1.5081467628479004


...

[INFO 05:38] t : 0, size : torch.Size([2447])
[INFO 05:38] t : 1, size : torch.Size([4916])
[INFO 05:38] t : 2, size : torch.Size([7448])
[INFO 05:38] t : 3, size : torch.Size([9897])
[INFO 05:38] 0.474661648273468, 1.45882248878479


100%|██████████| 4/4 [00:25<00:00,  6.42s/it]/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:144: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")


[INFO 05:44] t : 0, size : torch.Size([2476])
[INFO 05:44] t : 1, size : torch.Size([4964])
[INFO 05:44] t : 2, size : torch.Size([7458])
[INFO 05:45] t : 3, size : torch.Size([9961])
[INFO 05:45] 0.47485920786857605, 1.4637500047683716


5it [00:32,  6.44s/it]                       

[INFO 05:45] -- END Training --


In [ ]:
import numpy as np

def split_by_u_q_optimized(u, q, random_state=None):
    """
    Splits data tuples into two groups (e.g., query and meta) without separating tuples
    that share the same (u, q) pair.

    Optimized version: sorts the data by u so that each user's indices are contiguous,
    then processes each block to assign a random group label per unique q.

    Parameters
    ----------
    u : array-like
        Array of user values.
    q : array-like
        Array of question values.
    random_state : int or None, optional
        Seed for reproducibility.

    Returns
    -------
    groups : np.ndarray
        Array of group labels (0 or 1) for each data tuple.
    """
    # Convert to numpy arrays
    u = np.asarray(u)
    q = np.asarray(q)
    groups = np.empty(u.shape[0], dtype=np.int8)

    rng = np.random.default_rng(random_state)

    # Sort indices by u so that rows for the same user are contiguous.
    sorted_u_idx = np.argsort(u)
    u_sorted = u[sorted_u_idx]

    # Find the boundaries for each unique user.
    unique_users, start_idx = np.unique(u_sorted, return_index=True)
    end_idx = np.append(start_idx[1:], len(u_sorted))

    # Process each contiguous block corresponding to a single user.
    for s, e in zip(start_idx, end_idx):
        block_idx = sorted_u_idx[s:e]  # indices for one user
        q_block = q[block_idx]
        # For this user's block, get unique q values and an inverse index
        unique_q, inv = np.unique(q_block, return_inverse=True)
        # Randomly assign a group label (0 or 1) for each unique q in this block
        group_assignments = rng.choice([0, 1], size=len(unique_q), p=[0.8,0.2])
        # Map the group labels back to the original indices in the block
        groups[block_idx] = group_assignments[inv]

    return groups

# Example usage:
# Let's assume train[0] is a structured array/dict-like with keys 'user_id' and 'item_id'.
# For demonstration, we simulate some sample data:

import time
start = time.time()

for i in range(1):
    group_labels = split_by_u_q_optimized(train[0]['user_id'], train[0]['item_id'], random_state=42)

    # Now, all tuples with the same (user_id, item_id) pair have the same group label.
    query_mask = (group_labels == 0)
    meta_mask  = (group_labels == 1)

    query_users = train[0]['user_id'][query_mask].to_numpy()
    query_questions = train[0]['item_id'][query_mask].to_numpy()
    meta_users = train[0]['user_id'][meta_mask].to_numpy()
    meta_questions = train[0]['item_id'][meta_mask].to_numpy()
    #print(s2)
print("Total time:", time.time() - start)
